In [2]:
import pandas as pd
import numpy as np
 
# identify name of xlsx file (which will change when uploaded)
xlsx_filename = "Online Retail.xlsx"
 
# schema of the excel spreadsheet data range
orders_schema = {
  'InvoiceNo':str,
  'StockCode':str,
  'Description':str,
  'Quantity':np.int64,
  'InvoiceDate':np.datetime64,
  'UnitPrice':np.float64,
  'CustomerID':str,
  'Country':str  
  }
 
# read spreadsheet to pandas dataframe
# the xlrd library must be installed for this step to work 
orders_pd = pd.read_excel(
  xlsx_filename, 
  sheet_name='Online Retail',
  header=0, # first row is header
  dtype=orders_schema
  )
 
# display first few rows from the dataset
orders_pd.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom


In [3]:
import lifetimes
 
# set the last transaction date as the end point for this historical dataset
current_date = orders_pd['InvoiceDate'].max()
 
# calculate the required customer metrics
metrics_pd = (
  lifetimes.utils.summary_data_from_transaction_data(
    orders_pd,
    customer_id_col='CustomerID',
    datetime_col='InvoiceDate',
    observation_period_end = current_date, 
    freq='D'
    )
  )
 
# display first few rows
metrics_pd.head(10)

,frequency,recency,T
CustomerID,,,
12346,0.0,0.0,325.0
12347,6.0,365.0,367.0
12348,3.0,283.0,358.0
12349,0.0,0.0,18.0
12350,0.0,0.0,310.0
12352,6.0,260.0,296.0
12353,0.0,0.0,204.0
12354,0.0,0.0,232.0
12355,0.0,0.0,214.0


In [4]:
# summary data from lifetimes
metrics_pd.describe()

,frequency,recency,T
count,4372.000000,4372.000000,4372.000000
mean,3.413541,133.723010,225.304209
std,6.674343,133.000474,118.384168
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,115.000000
50%,1.000000,98.000000,253.000000
75%,4.000000,256.000000,331.000000
max,145.000000,373.000000,373.000000


In [6]:
from datetime import timedelta
 
# set the last transaction date as the end point for this historical dataset
current_date = orders_pd['InvoiceDate'].max()
 
# define end of calibration period
holdout_days = 90
calibration_end_date = current_date - timedelta(days = holdout_days)
 
# calculate the required customer metrics
metrics_cal_pd = (
  lifetimes.utils.calibration_and_holdout_data(
    orders_pd,
    customer_id_col='CustomerID',
    datetime_col='InvoiceDate',
    observation_period_end = current_date,
    calibration_period_end=calibration_end_date,
    freq='D'    
    )
  )
 
# display first few rows
metrics_cal_pd.head(10)

,frequency_cal,recency_cal,T_cal,frequency_holdout,duration_holdout
CustomerID,,,,,
12346,0.0,0.0,235.0,0.0,90.0
12347,4.0,238.0,277.0,2.0,90.0
12348,2.0,110.0,268.0,1.0,90.0
12350,0.0,0.0,220.0,0.0,90.0
12352,3.0,34.0,206.0,3.0,90.0
12353,0.0,0.0,114.0,0.0,90.0
12354,0.0,0.0,142.0,0.0,90.0
12355,0.0,0.0,124.0,0.0,90.0
12356,1.0,80.0,235.0,1.0,90.0


In [9]:
# remove customers with no repeats (complete dataset)
filtered_pd = metrics_pd[metrics_pd['frequency'] > 0]
 
## remove customers with no repeats in calibration period
filtered_cal = metrics_cal_pd[metrics_cal_pd['frequency_cal'] > 0]


In [11]:
from lifetimes.fitters.pareto_nbd_fitter import ParetoNBDFitter
from lifetimes.fitters.beta_geo_fitter import BetaGeoFitter
 
# load spark dataframe to pandas dataframe
input_pd = filtered_cal
 
# fit a model
model = ParetoNBDFitter(penalizer_coef=0.0)
model.fit( input_pd['frequency_cal'], input_pd['recency_cal'], input_pd['T_cal'])

C:\Users\Vivek Ketha\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Vivek Ketha\anaconda3\lib\site-packages\lifetimes\fitters\pareto_nbd_fitter.py:216: RuntimeWarning: invalid value encountered in logaddexp
  A_2 = logaddexp(-(r + x) * log(alpha + T) - s * log(beta + T), log(s) + log_A_0 - log(r_s_x))


<lifetimes.ParetoNBDFitter: fitted with 2163 subjects, alpha: 96.99, beta: 503893.36, r: 2.00, s: 137.56>

In [12]:
# get predicted frequency during holdout period
frequency_holdout_predicted = model.predict( input_pd['duration_holdout'], input_pd['frequency_cal'], input_pd['recency_cal'], input_pd['T_cal'])
 
# get actual frequency during holdout period
frequency_holdout_actual = input_pd['frequency_holdout']

In [13]:
import numpy as np
 
def score_model(actuals, predicted, metric='mse'):
  # make sure metric name is lower case
  metric = metric.lower()
  
  # Mean Squared Error and Root Mean Squared Error
  if metric=='mse' or metric=='rmse':
    val = np.sum(np.square(actuals-predicted))/actuals.shape[0]
    if metric=='rmse':
        val = np.sqrt(val)
  
  # Mean Absolute Error
  elif metric=='mae':
    val = np.sum(np.abs(actuals-predicted))/actuals.shape[0]
  
  else:
    val = None
  
  return val
 
# score the model
print('MSE: {0}'.format(score_model(frequency_holdout_actual, frequency_holdout_predicted, 'mse')))

MSE: 4.0472381253783825


In [14]:
from hyperopt import hp, fmin, tpe, rand, SparkTrials, STATUS_OK, STATUS_FAIL, space_eval
 
# define search space
search_space = hp.choice('model_type',[
                  {'type':'Pareto/NBD', 'l2':hp.uniform('pareto_nbd_l2', 0.0, 1.0)},
                  {'type':'BG/NBD'    , 'l2':hp.uniform('bg_nbd_l2', 0.0, 1.0)}  
                  ]
                )
 
# define function for model evaluation
def evaluate_model(params):
  
  # accesss replicated input_pd dataframe
  data = inputs.value
  
  # retrieve incoming parameters
  model_type = params['type']
  l2_reg = params['l2']
  
  # instantiate and configure the model
  if model_type == 'BG/NBD':
    model = BetaGeoFitter(penalizer_coef=l2_reg)
  elif model_type == 'Pareto/NBD':
    model = ParetoNBDFitter(penalizer_coef=l2_reg)
  else:
    return {'loss': None, 'status': STATUS_FAIL}
  
  # fit the model
  model.fit(data['frequency_cal'], data['recency_cal'], data['T_cal'])
  
  # evaluate the model
  frequency_holdout_actual = data['frequency_holdout']
  frequency_holdout_predicted = model.predict(data['duration_holdout'], data['frequency_cal'], data['recency_cal'], data['T_cal'])
  mse = score_model(frequency_holdout_actual, frequency_holdout_predicted, 'mse')
  
  # return score and status
  return {'loss': mse, 'status': STATUS_OK}

In [16]:
!pip install mlflow

     --------------------------------------- 16.7/16.7 MB 11.7 MB/s eta 0:00:00
     -------------------------------------- 147.5/147.5 kB 8.6 MB/s eta 0:00:00
  Using cached sqlparse-0.4.3-py3-none-any.whl (42 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ------------------------------------- 210.6/210.6 kB 13.4 MB/s eta 0:00:00
     ---------------------------------------- 82.3/82.3 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 435.6/435.6 kB 9.0 MB/s eta 0:00:00
     ---------------------------------------- 57.7/57.7 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB ? eta 0:00:00
     ---------------------------------------- 62.8/62.8 kB 3.3 MB/s eta 0:00:00
  Using cached urllib3-1.26.14-py2.py3-none-any.whl (140 kB)
     --------------------------------------- 12.2/12.2 MB 10.7 MB/s eta 0:00:00
  Created wheel

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
snowflake-connector-python-lite 0.0.1 requires requests==2.22.0, but you have requests 2.28.2 which is incompatible.
snowflake-connector-python-lite 0.0.1 requires urllib3<1.26.0,>=1.20, but you have urllib3 1.26.14 which is incompatible.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.20 requires requests==2.28.1, but you have requests 2.28.2 which is incompatible.
alchemy 20.5 requires filelock==3.0.12, but you have filelock 3.9.0 which is incompatible.
alchemy 20.5 requires requests==2.22.0, but you have requests 2.28.2 which is incompatible.


In [18]:
# get hyperparameter settings
params = space_eval(search_space, argmin)
model_type = params['type']
l2_reg = params['l2']
 
# instantiate and configure model
if model_type == 'BG/NBD':
  model = BetaGeoFitter(penalizer_coef=l2_reg)
elif model_type == 'Pareto/NBD':
  model = ParetoNBDFitter(penalizer_coef=l2_reg)
else:
  raise 'Unrecognized model type'
  
# train the model
model.fit(input_pd['frequency_cal'], input_pd['recency_cal'], input_pd['T_cal'])

NameError: name 'argmin' is not defined

In [19]:
# score the model
frequency_holdout_actual = input_pd['frequency_holdout']
frequency_holdout_predicted = model.predict(input_pd['duration_holdout'], input_pd['frequency_cal'], input_pd['recency_cal'], input_pd['T_cal'])
mse = score_model(frequency_holdout_actual, frequency_holdout_predicted, 'mse')
 
print('MSE: {0}'.format(mse))

MSE: 4.0472381253783825


In [20]:
# add a field with the probability a customer is currently "alive"
filtered_pd['prob_alive']=model.conditional_probability_alive(
    filtered_pd['frequency'], 
    filtered_pd['recency'], 
    filtered_pd['T']
    )
 
filtered_pd.head(10)

C:\Users\Vivek Ketha\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


,frequency,recency,T,prob_alive
CustomerID,,,,
12347,6.0,365.0,367.0,0.999445
12348,3.0,283.0,358.0,0.967933
12352,6.0,260.0,296.0,0.985444
12356,2.0,303.0,325.0,0.993338
12358,1.0,149.0,150.0,0.999725
12359,5.0,324.0,331.0,0.997978
12360,2.0,148.0,200.0,0.979170
12362,12.0,292.0,295.0,0.999136
12363,1.0,133.0,242.0,0.947727


In [21]:
filtered_pd['purchases_next30days']=(
  model.conditional_expected_number_of_purchases_up_to_time(
    30, 
    filtered_pd['frequency'], 
    filtered_pd['recency'], 
    filtered_pd['T']
    )
  )
 
filtered_pd.head(10)

C:\Users\Vivek Ketha\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Vivek Ketha\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in logaddexp
  result = getattr(ufunc, method)(*inputs, **kwargs)


,frequency,recency,T,prob_alive,purchases_next30days
CustomerID,,,,,
12347,6.0,365.0,367.0,0.999445,0.514602
12348,3.0,283.0,358.0,0.967933,0.317554
12352,6.0,260.0,296.0,0.985444,0.599060
12356,2.0,303.0,325.0,0.993338,0.281044
12358,1.0,149.0,150.0,0.999725,0.362325
12359,5.0,324.0,331.0,0.997978,0.487400
12360,2.0,148.0,200.0,0.979170,0.393635
12362,12.0,292.0,295.0,0.999136,1.065854
12363,1.0,133.0,242.0,0.947727,0.250262


In [24]:
frequency = 6
recency = 255
T = 300
t = 30
 
print('Probability of Alive: {0}'.format( model.conditional_probability_alive(frequency, recency, T) ))
print('Expected Purchases in next {0} days: {1}'.format(t, model.conditional_expected_number_of_purchases_up_to_time(t, frequency, recency, T) ))

Probability of Alive: 0.9798284205005244
Expected Purchases in next 30 days: 0.5896446083993535
